# Exercícios de Fixação: Filter, Map e Reduce

Todos as próximas questÕes deverão usar a função `mostra_dados(titulo, nome_arq, func)` para testar as funções criadas.

In [11]:
import csv
from functools import reduce

def mostra_dados(titulo, nome_arq, f, *args):
    """
    Função auxiliar para mostrar o resultado da execução de uma função do paradigma funcional.
    A função abre um arquivo CSV e passa para função do paradigma funcional um objeto DictReader e 
    depois mostra o resultado na tela.
    """
    print("|___   ", titulo, "  ___|")
    with open(nome_arq, "r", encoding='utf-8-sig') as arq:
        dados = csv.DictReader(arq, delimiter=',')
        fdados = f(dados, *args)
        try:
            iterator = iter(fdados)
        except TypeError:
            print(fdados)
        else:
            for linha in fdados:
                print(linha)
                
file_path = "../../brasi2000-2021.csv"

1. Escreva uma função pura `add_campo_ano`, que adicione um novo campo `ano` para cada dicionário (cada dicionário é um jogo do campeonato) lido via objeto `DictReader`

In [ ]:
def add_campo_ano(dados):
    return map(lambda x: {**x, "ano": x["Data"][:4]}, dados)

mostra_dados("Mostrando os dados do arquivo", file_path, add_campo_ano)

2. Escreva uma função pura `add_total_gols`, que adicione um novo campo `total_gols` para cada dicionário (cada dicionário é um jogo do campeonato) lido via objeto `DictReader`

In [ ]:
def add_total_gols(dados):
    return map(lambda x: {**x, "Total de Gols": int(x["Mandante Placar"]) + int(x["Visitante Placar"])}, dados)

mostra_dados("Mostrando os dados do arquivo", file_path, add_total_gols)

3. Escreva uma função pura `contar_gols`, que conte o número de gols que foram marcados em todos os jogos de uma estrutura iterável, ou seja, de todo o campeonato brasileiro.
    1. Escreva uma função `jogo_em_gols` que mapeia a sequência de jogos em uma lista de gols

In [ ]:
from functools import reduce

def jogo_em_gols(dados):
    return map(lambda x: int(x["Mandante Placar"]) + int(x["Visitante Placar"]), dados)

def soma_jogos(dados):
    return reduce(lambda x, y: x + y, dados)

mostra_dados("Mostrando os dados do arquivo", "../brasi2000-2021.csv", lambda x: soma_jogos(jogo_em_gols(x)))

4. Escreva uma função pura `filtra_ano`, que filtra/seleciona todos os jogos referente a um determinado ano

In [ ]:
def filtra_ano(ano):
    def filtra(dados):
        return filter(lambda x: int(x["Data"][:4]) == ano, dados)
    return filtra

mostra_dados("Mostrando os dados do arquivo", "../brasi2000-2021.csv", filtra_ano(2001))

5. Escreva uma função pura `filtra_time`, que filtra/seleciona todos os jogos referente a um determinado time. Lembre que o time pode ser tanto o mandante quanto o visitante.

In [ ]:
def filtra_time(time):
    def filtra(dados):
        return filter(lambda x: x["Madante"] == time or x["Visitante"] == time, dados)
    return filtra

mostra_dados("Mostrando os dados do arquivo", "../brasi2000-2021.csv", filtra_ano(2001))

6. Escreva uma função pura `filtra_vitorias`, que filtra/seleciona todos os jogos referente a uma determinada equipe que venceu a partida. Lembre que o time pode ser tanto o mandante quanto o visitante. Utilize de curry para poder selecionar o time especifico.

In [ ]:
def filtra_vitorias(time):
    def filtra(dados):
        return filter(lambda x: x["Vencedor"] == time, dados)
    return filtra

mostra_dados("Mostrando os dados do arquivo", file_path, filtra_vitorias("Flamengo"))

7. Escreva uma função `conta_vitorias_time` que conta o número de vitórias de um time específico.

In [ ]:
def conta_vitorias_time(time):
    def filtra(dados):
        return reduce(lambda x,y: x + y, map(lambda x: 1, filter(lambda x: x["Vencedor"] == time, dados)))
    return filtra

mostra_dados("Mostrando os dados do arquivo", file_path, conta_vitorias_time("Flamengo"))

8. Escreva uma função pura `todas_equipes()` para mostrar todas as equipes que jogaram nas edições do campeonato brasileiro, entre 2000 e 2021. Crie uma função `obter_equipe()` para ser usada como acumulador na função `reduce`.
    
_Dica: usar estrutura de dados `set` para o acumulador e assim evitar duplicados_

In [ ]:
def todas_equipes(dados):
    return [" ".join(set(map(lambda x: x["Mandante"].lower(), dados)))]

mostra_dados("Mostrando os dados do arquivo", "../brasi2000-2021.csv", todas_equipes)

9. Escreva uma função pura `total_gols_equipe`, que encontra o número total de gols que foram marcados por uma dada equipe. Depois, criei uma closure para poder reusá-la para diferentes equipes.

In [ ]:
def total_gols_equipe(equipe):
    def total_gols(dados):
        return reduce(lambda x, y: x + int(y["Mandante Placar"]) + int(y["Visitante Placar"]), filter(lambda x: x["Mandante"].lower() == equipe, dados), 0)
    return total_gols

mostra_dados("Mostrando os dados do arquivo", "../brasi2000-2021.csv", total_gols_equipe("palmeiras"))

10. Escreva uma função pura `contar_gols_por_ano()` que retorna uma sequência de tuplas em que temos o ano (2000 até 2021) e o número de gols que aconteceram naquele ano. 

_Dica: Pode ser feito de duas formas, ou usando `map` ou `reduce`_

In [ ]:
def contar_gols_por_ano(dados):
    gols_por_ano = reduce(
        lambda acc, jogo: {
            **acc, 
            int(jogo["Data"][:4]): acc.get(int(jogo["Data"][:4]), 0) + int(jogo["Mandante Placar"]) + int(jogo["Visitante Placar"])
        }, 
        dados, 
        {}
    )
    return [(ano, gpa) for ano, gpa in gols_por_ano.items()]



mostra_dados("Mostrando os dados do arquivo", "../brasi2000-2021.csv", contar_gols_por_ano)

11. Escreva uma função pura `total_gols_por_equipe`, que produza uma lista de tuplas `(equipe, total_gols)` com a equipe e o total de gols que ela marcou. Ordene esta lista em ordem descendente.

In [ ]:
def total_gols_por_equipe(dados):
    gols_por_equipe = reduce(
        lambda acc, jogo: {
            **acc,
            jogo['Mandante'].lower(): acc.get(jogo['Mandante'].lower(), 0) + int(jogo['Mandante Placar']),
            jogo['Visitante'].lower(): acc.get(jogo['Visitante'].lower(), 0) + int(jogo['Visitante Placar'])
        },
        dados,
        {}
    )
    
    return sorted(gols_por_equipe.items(), key=lambda x: x[1], reverse=True)

mostra_dados("Mostrando os dados do arquivo", "../brasi2000-2021.csv", total_gols_por_equipe)